<a href="https://colab.research.google.com/github/christinium/AIMed_Workshop_2018/blob/master/SQL_and_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Databases
### Adapated from HST953 2017 Course
## Objectives

In this section we will address:

 - An understanding of relational databases
 - Understanding of CSV format
 - Familiarity with the MIMIC-III database
 - Ability to select data from a database using Structured Query Language (SQL)



## Introduction to MIMIC-III

MIMIC-III is an openly available dataset developed by the MIT Lab for Computational Physiology, comprising deidentified health data associated with ~60,000 hospital stays. Spanning 2001-2012, it includes demographics, vital signs, laboratory tests, medications, and more. A paper describing MIMIC-III is available from: http://www.nature.com/articles/sdata201635

The dataset is provided as a collection of comma-separated value (CSV) files, which can be loaded into a database system such as PostgreSQL. A list of tables is provided on the MIMIC website: http://mimic.physionet.org/mimictables/admissions/

We have highlighted some of the key tables below:

- patients: a list of patients covered in MIMIC-III, each identified by a unique subject_id.
- admissions: a list of hospital admissions, each identified by a unique hadm_id.
- icustays: a list of ICU stays, each identified by a unique icustay_id.

Querybuilder (https://mimic.physionet.org/gettingstarted/querybuilder/) allows you to explore MIMIC-III with simple queries. 

For detailed analysis, we recommend that you build MIMIC-III in a Postgres database on your local computer by following the instructions on the MIMIC website (https://mimic.physionet.org/tutorials/install-mimic-locally-ubuntu/).


* Patient *
* Admissions *
* ICUstays *


## Comma separated value (CSV) files

Comma separated value (CSV) files are a plain text format used for storing data in a tabular, spreadsheet-style structure. While there is no hard and fast rule for structuring tabular data, it is usually considered good practice to include a header row, to list each variable in a separate column, and to list observations in rows.

As there is no official standard for the CSV format, the term is used somewhat loosely, which can often cause issues when seeking to load the data into a data analysis package. A general recommendation is to follow the definition for CSVs set out by the Internet Engineering Task Force in the RFC 4180 specification document.

<img src="https://stuff.mit.edu/~cwc76/colab/images_aimed_2018/csvformat.png">

Summarized briefly, RFC 4180 specifies that:

- files may optionally begin with a header row, with each field separated by a comma;
- records should be listed in subsequent rows. Fields should be separated by commas, and each row should be terminated with a line break;
- fields that contain numbers may be optionally enclosed within double quotes;
- fields that contain text ("strings") should be enclosed within double quotes;
- if a double quote appears inside a string of text then it must be escaped with a preceding double quote.

The CSV format is popular largely because of its simplicity and versatility. CSV files can be edited with a text editor, loaded as a spreadsheet in packages such as Microsoft Excel, and imported and processed by most data analysis packages. Often CSV files are an intermediate data format used to hold data that has been extracted from a relational database in preparation for analysis. 

## Relational Databases
Relational databases can be thought of as a collection of tables which are linked together by **shared keys**. Organizing data across tables can help to maintain data integrity and enable faster analysis and more efficient storage.

### Motivation: why would we want a relational database?

Imagine trying to store data about a person: their name, age, and height. We can easily save this data in a CSV:

```
"Name", "Age", "Height"
"Penny", 30, 182
```

Now what if we measure Penny and Paul's heart rate every hour for four hours at 8:00am, 9:00am, 10:00am, and 11:00am. How should we store this data? The naive approach would be to simply concatenate the information we have all in one file:

```
"Name", "Age", "Height", "Time", "Heart rate"
"Penny", 30, 182, "8:00", 65
"Penny", 30, 182, "9:00", 71
"Penny", 30, 182, "10:00", 72
"Paul", "31", "185", "8:00", 88
"Paul", "31", "185", "9:00", 85
"Paul", "31", "185", "10:00", 80
```

This works, but it feels very inefficient. We have repeated her name ("Penny"), her age (30), and her height (182) every time we get a heart rate measurement.

The immediate solution is to not store both of these in the same file: we make one file for demographics (age, height), and we make another file for heart rate measurements. For fun we will add in a unique ID number in case there are two people with the same name.Then, we make sure that the ID is same in both, so that we know who has which heart rates.


**Table Demographics**
```
"ID", Name", "Age", "Height"
1, "Penny", 30, 182
2, "Paul", "31", "185"
```
**Table Heart Rate**
```
"ID", "Time", "Heart rate"
1, "8:00", 65
1, "9:00", 71
1, "10:00", 72
2, "8:00", 88
2, "9:00", 85
2, "10:00", 80
```

We've created a relational database. Since the ID is what links the two tables together, we would call the name column a "key".

## Terminology

- **"Database schema":** The model that defines the structure and relationships of the tables.
- **"Database query":** Data is extracted from relational databases using structured "queries".
- **"Primary key":** A primary key is a field that uniquely identifies each row in a table.
- **"Foreign key"**: A foreign key is a field that refers to a primary key in another table.
- **"Normalisation":** The concept of structuring a database in a way that reduces data repetition and improves data integrity, usually by requiring one or more tables to be joined.
- **"Denormalisation":** The concept of structuring a database to improve readability, sometimes at the expense of data repetition and data integrity.
-**"Data type":** A term used to describe the behaviour of data and the possible values that it can hold (for example, integer, text, and date are all data types in PostgreSQL).

Giving a simple example of a hospital database with four tables, it might comprise of:

- Table 1: **patients**, a list of all patients;
- Table 2, **admissions**, a list of all hospital admissions;
- Table 3, **chartevents**, a list of vital sign measurements;
- Table 4, **d_items**, a dictionary of vital sign codes and associated labels.


<img src="https://stuff.mit.edu/~cwc76/colab/images_aimed_2018/relationaldb.png">


## What is SQL

Structured Query Language (SQL) is a programming language used to manage relational databases. An SQL query has the following format:

```
SELECT [columns]
FROM [database_name].[table_name];
```

The result of a query is generally a list of rows selected from your table/s of interest. For example, the following query lists the unique patient identifiers (subject_ids) and gender of everyone in the patients table:

```
SELECT subject_id, gender
FROM mimiciii.patients;
```

The asterisk (*) character is a wildcard that can be used to select all columns.

```
SELECT *
FROM mimiciii.patients;
```

Note, in PostgreSQL we can avoid specifying the database name in our queries by setting the search path:

```
SET SEARCH_PATH TO mimiciii;
```

**Questions:**
1) Write SQL that selects all the data from the patients table
2) Write SQL that only selects the subject_id, dob, and gender columns from the patients table


## SQL:WHERE keyword
Often you will want to select a subset of the data which satisfy some set of conditions. For example, you may want to select only female subjects from the database. This is easily accomplished with the WHERE keyword. The framework of our query becomes:
```
SELECT [columns]
FROM [table_name]
WHERE [conditions];
```
We can easily select all the subject_id corresponding to female subjects as follows:

```
SELECT subject_id
FROM patients
WHERE gender = 'F';
```

WHERE clauses are used to make a query return rows meeting only our specified criteria (our previous query, for example, returning only female patients). The simplest criteria is equality, WHERE gender = 'F'. Note that in this situation we specify a string, but this syntax will work for numbers as well. For example, we could select all the data for a single subject:

```
SELECT *
FROM patients
WHERE subject_id = 252;
```

WHERE clauses can be combined with standard logical operators AND/OR:

```
SELECT *
FROM patients
WHERE subject_id = 253
OR subject_id = 254
OR subject_id = 255;
```

A useful shorthand for OR statements on the same column is the IN condition:

```
SELECT *
FROM patients
WHERE subject_id IN (253, 254, 255);
```

We can also use the "less than" (<), "less than or equal to" <=, "greater than" (>), or "greater than or equal to" >= operators:

```
SELECT *
FROM patients
WHERE subject_id >= 253
AND subject_id <= 255;
```

SQL also offers shorthand for >= and <= combinations with the BETWEEN condition:

```
SELECT *
FROM patients
WHERE subject_id BETWEEN 253 AND 255;
```

Note the BETWEEN operator is inclusive. Verify for yourself that the above two queries give the same result. When working with text data, we'll often want to search for partial string matches rather than exact matches. This can be accomplished with the LIKE keyword:

- use `LIKE` to match text
- The % is a wildcard that will match all characters

```
SELECT *
FROM microbiologyevents as micro
WHERE org_name like '%STREP%';
```

Note the use of the wildcard character %.

## ORDER BY keyword

The ORDER BY keyword is relatively straightforward: it will sort the data in the order you specify.

```
SELECT [columns]
FROM [table_name]
WHERE [conditions]
ORDER BY [columns] [ASC/DESC];
```

The query below orders the results by the patient dob

```
SELECT subject_id, dob
FROM patients
ORDER BY dob;
```

Note that the WHERE clause is optional and in the query above it has been omitted. However, we must respect the order of the keywords. If we use the WHERE keyword it must appear after the FROM keyword and before the ORDER BY keyword.

## Using SQL JOIN to query multiple tables
Often we need information coming from multiple tables. This can be achieved using the SQL JOIN keyword. There are several types of join, including INNER JOIN, LEFT JOIN, and RIGHT JOIN. It is important to understand the difference between these joins because their usage can significantly impact query results. Detailed guidance on joins is widely available on the web.

<img src="https://stuff.mit.edu/~cwc76/colab/images_aimed_2018/sql-joins.png">

Using the INNER JOIN keyword, let’s select a list of patients from the patients table along with dates of birth, and join to the admissions table to get the admission time for each hospital admission. We use the INNER JOIN to indicate that two or more tables should be combined based on a common attribute, which in our case is subject_id:

```
-- INNER JOIN will only return rows where subject_id
-- appears in both the patients table and the admissions table
SELECT p.subject_id, p.dob, a.hadm_id, a.admittime
FROM patients p
INNER JOIN admissions a
ON p.subject_id = a.subject_id
ORDER BY subject_id, hadm_id;
```

## Additional Information

Shared code for MIMIC-III (and eICU) can be found on github:

https://github.com/MIT-LCP/mimic-code

https://github.com/MIT-LCP/eicu-code
